In [3]:
import pandas as pd
import numpy as np
from openai import OpenAI
import openai
import warnings
from dotenv import load_dotenv
import os
import re

warnings.filterwarnings("ignore")
load_dotenv()
openai.api_key = os.environ.get("REASONING_API_KEY")
if not openai.api_key:
    raise ValueError("API Key for OpenAI not set!")
client = OpenAI(api_key=os.environ.get("REASONING_API_KEY"))

In [29]:
def query_gpt(prompt):
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}
]
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages = message,
        max_tokens=1024,
        temperature=0.7,
        n=1,
        stop=None,
    )

    return response.choices[0].message.content

In [110]:
df = pd.read_csv('../embeddings/data/113_gsm8k_gpt35_cot_responses.csv')
import ast
df['CoT Steps'] = df['CoT Steps'].apply(ast.literal_eval)

In [49]:
df

,Problem Number,Question,Answer,CoT Response,CoT Sentences
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal..."
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ..."
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett..."
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie..."
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja..."
...,...,...,...,...,...
108,108,Simon wanted to buy flowers that his mom could...,5 pansies at $2.50 each is 5*2.50 = $<<5*2.5=1...,"First, let's calculate the total cost of the f...","**\nFirst, let's calculate the total cost of t..."
109,109,Derek has $960 to buy his books for the semest...,Amount spent on textbooks is 960/2 = <<960/2=4...,"First, let's calculate how much Derek spends o...","**\nFirst, let's calculate how much Derek spen..."
110,110,Caleb and his dad went fishing at the lake. C...,Caleb’s dad caught 2 x 3 = <<2*3=6>>6 trouts.\...,Step 1: Caleb caught 2 trouts.\nStep 2: Caleb'...,**\nStep 1: Caleb caught 2 trouts.\n\n||\nStep...
111,111,Christina has 3 snakes. 1 snake is 2 feet long...,The first snake is 24 inches because there are...,"First, let's convert the measurements to the s...","**\nFirst, let's convert the measurements to t..."


In [50]:
def split_cot(text, delimiters=["||", "**"]):
    if not isinstance(text, str):
        return None
    
    delimiters = [re.escape(delimiter) for delimiter in delimiters]
    pattern = '|'.join(delimiters)
    split_text = re.split(pattern, text)
    formatted_text = []
    for section in split_text:
        section = section.replace('\n', ' ').strip()
        if section:
            formatted_text.append(section)
    return formatted_text

df['CoT Steps'] = df['CoT Sentences'].apply(split_cot)

In [71]:
df.to_csv('../embeddings/data/113_gsm8k_gpt35_cot_responses.csv', index=False)

In [111]:
df

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","[To find out how much Weng earned, we need to ...",train
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[First, let's calculate how much money Betty h...",train
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie...","[First, let's find out how many pages Julie ha...",train
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja...","[First, let's calculate how many letters James...",train
...,...,...,...,...,...,...,...
108,108,Simon wanted to buy flowers that his mom could...,5 pansies at $2.50 each is 5*2.50 = $<<5*2.5=1...,"First, let's calculate the total cost of the f...","**\nFirst, let's calculate the total cost of t...","[First, let's calculate the total cost of the ...",val
109,109,Derek has $960 to buy his books for the semest...,Amount spent on textbooks is 960/2 = <<960/2=4...,"First, let's calculate how much Derek spends o...","**\nFirst, let's calculate how much Derek spen...","[First, let's calculate how much Derek spends ...",val
110,110,Caleb and his dad went fishing at the lake. C...,Caleb’s dad caught 2 x 3 = <<2*3=6>>6 trouts.\...,Step 1: Caleb caught 2 trouts.\nStep 2: Caleb'...,**\nStep 1: Caleb caught 2 trouts.\n\n||\nStep...,"[Step 1: Caleb caught 2 trouts., Step 2: Caleb...",val
111,111,Christina has 3 snakes. 1 snake is 2 feet long...,The first snake is 24 inches because there are...,"First, let's convert the measurements to the s...","**\nFirst, let's convert the measurements to t...","[First, let's convert the measurements to the ...",val


In [56]:
df = df.drop(["CoT Sentences"], axis=1)

In [59]:
split = ['train'] * 90 + ['val'] * 23

In [58]:
df.loc[df['Problem Number'] < 90]

,Problem Number,Question,Answer,CoT Response,CoT Steps
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","[""First, let's find out how many clips Natalia..."
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","['To find out how much Weng earned, we need to..."
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","[""First, let's calculate how much money Betty ..."
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","[""First, let's find out how many pages Julie h..."
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","[""First, let's calculate how many letters Jame..."
...,...,...,...,...,...
85,85,Toby is counting goldfish in the local pond. H...,There are 60 goldfish because 15 / .25 = <<15/...,Step 1: Calculate the percentage of goldfish b...,['Step 1: Calculate the percentage of goldfish...
86,86,James watched 2 episodes of Jeopardy and 2 epi...,He watched 2*20=<<2*20=40>>40 minutes of Jeopa...,"First, let's calculate the total time James sp...","[""First, let's calculate the total time James ..."
87,87,Winwin won $50 in a lottery. She paid 20% for ...,Winwin paid $50 x 20/100 = $<<50*20/100=10>>10...,Step 1: Calculate the tax amount.\nTo find the...,['Step 1: Calculate the tax amount. To find th...
88,88,Angela is a bike messenger in New York. She ne...,Let p be the number of packages Angela deliver...,"First, let's assume that Angela delivers x mea...","[""First, let's assume that Angela delivers x m..."


In [69]:
df['split'] = split

In [73]:
train_df = df.loc[df.split == 'train']
val_df = df.loc[df.split == 'val']

In [74]:
train_df

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","[To find out how much Weng earned, we need to ...",train
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[First, let's calculate how much money Betty h...",train
3,3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....,"First, let's find out how many pages Julie has...","**\nFirst, let's find out how many pages Julie...","[First, let's find out how many pages Julie ha...",train
4,4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,"First, let's calculate how many letters James ...","**\nFirst, let's calculate how many letters Ja...","[First, let's calculate how many letters James...",train
...,...,...,...,...,...,...,...
85,85,Toby is counting goldfish in the local pond. H...,There are 60 goldfish because 15 / .25 = <<15/...,Step 1: Calculate the percentage of goldfish b...,**\nStep 1: Calculate the percentage of goldfi...,[Step 1: Calculate the percentage of goldfish ...,train
86,86,James watched 2 episodes of Jeopardy and 2 epi...,He watched 2*20=<<2*20=40>>40 minutes of Jeopa...,"First, let's calculate the total time James sp...","**\nFirst, let's calculate the total time Jame...","[First, let's calculate the total time James s...",train
87,87,Winwin won $50 in a lottery. She paid 20% for ...,Winwin paid $50 x 20/100 = $<<50*20/100=10>>10...,Step 1: Calculate the tax amount.\nTo find the...,**\nStep 1: Calculate the tax amount.\nTo find...,[Step 1: Calculate the tax amount. To find the...,train
88,88,Angela is a bike messenger in New York. She ne...,Let p be the number of packages Angela deliver...,"First, let's assume that Angela delivers x mea...","**\nFirst, let's assume that Angela delivers x...","[First, let's assume that Angela delivers x me...",train


In [75]:
def query_gpt(prompt):
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}

]
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages = message,
        max_tokens=1024,
        temperature=0.0,
        n=1,
        stop=None,
    )

    return response.choices[0].message.content

In [87]:
question = df['Question'].tolist()[1]

In [88]:
steps = df['CoT Steps'].tolist()[1]
steps

['To find out how much Weng earned, we need to calculate her earnings per minute and then multiply it by the number of minutes she worked.  First, we need to convert the hourly rate to a minute rate. Since there are 60 minutes in an hour, we divide $12 by 60 to find out how much Weng earns per minute.  $12 / 60 minutes = $0.2 per minute',
 'Now, we can multiply the minute rate by the number of minutes Weng worked to find out her earnings.  $0.2 per minute * 50 minutes = $10',
 'Therefore, Weng earned $10 for 50 minutes of babysitting.']

In [91]:
prompt = f'''
Given a math word problem and a step-by-step explanation of the solution, reword the question and the steps of the answer to involve a different "cover story" e.g. change any references to objects, names, actions, months, places etc.
Include the reworded question and steps as a JSON, 

**
Question:
**
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
**
Steps:
**
["First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.", 'To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72', 'Therefore, Natalia sold a total of 72 clips in April and May.']
**
Reworded Question:
**
Marco baked cookies for 48 of his neighbors in January, and then he baked half as many cookies in February. How many cookies did Marco bake in total during January and February?
**
Reworded Steps:
["First, we need to determine the quantity of cookies Marco baked in February. It is given that he baked half the amount in February compared to January. Given that he baked 48 cookies in January, the number of cookies he baked in February is 48 divided by 2, which equals 24 cookies.", "To calculate the total number of cookies Marco baked over both January and February, we sum up the cookies from January with the cookies from February. That's 48 cookies for January plus 24 cookies for February, yielding a total of 48 + 24 = 72 cookies.", "Hence, Marco baked a combined total of 72 cookies in the months of January and February."]
**

**
Question: 
**
{question}
**
Steps:
**
{steps}
'''
print(prompt)


Given a math word problem and a step-by-step explanation of the solution, reword the question and the steps of the answer to involve a different "cover story" e.g. change any references to objects, names, actions, months, places etc.
Include the reworded question and steps as a JSON, 

**
Question:
**
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
**
Steps:
**
["First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.", 'To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72', 'Therefore, Natalia sold a total of 72 clips in April and May.']
**
Reworded Question:
**
Marco baked cookies for 48 of his neighbors in January, and then h

In [104]:
prompt = f'''
Given a math word problem and a step-by-step explanation of the solution, reword the question and the steps of the answer to involve a different "cover story" e.g. change any references to objects, names, actions, months, places etc.
Include the reworded question and steps as a JSON. Each element in the steps array is a step; ensure that the number of steps in reworded_steps equals the number of steps in the original steps.

{{
  "original_question": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
  "original_steps": ["First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.", 'To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72', 'Therefore, Natalia sold a total of 72 clips in April and May.']
}}

{{
  "reworded_question": "Marco baked cookies for 48 of his neighbors on Monday, and then he baked half as many cookies on Tuesday. How many cookies did Marco bake in total on Monday and Tuesday?",
  "reworded_steps": ["First, we need to determine the number of cookies Marco baked on Tuesday. It is given that he baked half the amount on Tuesday compared to Monday. Given that he baked 48 cookies on Monday, the number of cookies he baked on Tuesday is 48 divided by 2, which equals 24 cookies.", "To calculate the total number of cookies Marco baked on both Monday and Tuesday, we sum the number of cookies from Monday and Tuesday. That's 48 cookies for Monday plus 24 cookies for Tuesday, yielding a total of 48 + 24 = 72 cookies.", "Hence, on Monday and Tuesday, Marco baked a combined total of 72 cookies."]
}}

{{
  "original_question": {question},
  "original_steps": {steps}
}}
'''
print(prompt)


Given a math word problem and a step-by-step explanation of the solution, reword the question and the steps of the answer to involve a different "cover story" e.g. change any references to objects, names, actions, months, places etc.
Include the reworded question and steps as a JSON. Each element in the steps array is a step; ensure that the number of steps in reworded_steps equals the number of steps in the original steps.

{
  "original_question": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
  "original_steps": ["First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.", 'To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 

In [95]:
reworded = query_gpt(prompt)
print(reworded)

```json
{
  "reworded_question": "Liam gets paid $12 for each lawn he mows. This weekend, he mowed lawns for 50 minutes. How much money did Liam make?",
  "reworded_steps": [
    "First, we need to figure out Liam's earnings per minute, then multiply that by the time spent mowing. We begin by converting the pay per lawn into a pay per minute rate. With 60 minutes in an hour, we take his pay of $12 and divide by 60 to find his earnings per minute. $12 / 60 minutes = $0.20 per minute.",
    "Next, we multiply the per-minute rate by the minutes Liam spent mowing. $0.20 per minute * 50 minutes equals $10.",
    "In conclusion, Liam made $10 from mowing lawns for 50 minutes."
  ]
}
```


In [97]:
def gpt_query_json(prompt):
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}

]
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages = message,
        response_format={"type": "json_object"},
        max_tokens=1024,
        temperature=0.0,
        n=1,
        stop=None,
    )

    return response.choices[0].message.content

In [98]:
reworded_json = gpt_query_json(prompt)
print(reworded_json)

{
  "reworded_question": "Liam gets paid $12 for each lawn he mows. He mowed lawns for 50 minutes yesterday. How much did he make in that time?",
  "reworded_steps": [
    "First, we need to figure out Liam's earnings per minute, then multiply that by the time he spent mowing lawns. We start by converting his pay per lawn into a pay per minute rate. Since there are 60 minutes in an hour, we divide $12 by 60 to find out Liam's earnings per minute. $12 / 60 minutes = $0.2 per minute.",
    "Next, we multiply the per-minute rate by the number of minutes Liam spent mowing lawns to calculate his total earnings. $0.2 per minute * 50 minutes = $10.",
    "Thus, Liam made $10 for mowing lawns for 50 minutes."
  ]
}


In [99]:
type(reworded_json)

str

In [100]:
import json

reworded_dict = json.loads(reworded_json)

# Access and print the values
print("Question:", reworded_dict["reworded_question"])
for step in reworded_dict["reworded_steps"]:
    print(step)

Question: Liam gets paid $12 for each lawn he mows. He mowed lawns for 50 minutes yesterday. How much did he make in that time?
First, we need to figure out Liam's earnings per minute, then multiply that by the time he spent mowing lawns. We start by converting his pay per lawn into a pay per minute rate. Since there are 60 minutes in an hour, we divide $12 by 60 to find out Liam's earnings per minute. $12 / 60 minutes = $0.2 per minute.
Next, we multiply the per-minute rate by the number of minutes Liam spent mowing lawns to calculate his total earnings. $0.2 per minute * 50 minutes = $10.
Thus, Liam made $10 for mowing lawns for 50 minutes.
Type of reworded_dict: <class 'dict'>


In [101]:
type(reworded_dict["reworded_steps"])

list

In [106]:
for i, row in df.sample(5).iterrows():
    question = row['Question']
    steps = row['CoT Steps']

    reword_prompt = f'''
        Given a math word problem and a step-by-step explanation of the solution, reword the question and the steps of the answer to involve a different "cover story" e.g. change any references to objects, names, actions, months, places etc.
        Include the reworded question and steps as a JSON. Each element in the steps array is a step; ensure that the number of steps in reworded_steps equals the number of steps in the original steps. Do not change the numbers in the problem or solution.

        {{
        "original_question": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
        "original_steps": ["First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.", 'To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72', 'Therefore, Natalia sold a total of 72 clips in April and May.']
        }}

        {{
        "reworded_question": "Marco baked cookies for 48 of his neighbors on Monday, and then he baked half as many cookies on Tuesday. How many cookies did Marco bake in total on Monday and Tuesday?",
        "reworded_steps": ["First, we need to determine the number of cookies Marco baked on Tuesday. It is given that he baked half the amount on Tuesday compared to Monday. Given that he baked 48 cookies on Monday, the number of cookies he baked on Tuesday is 48 divided by 2, which equals 24 cookies.", "To calculate the total number of cookies Marco baked on both Monday and Tuesday, we sum the number of cookies from Monday and Tuesday. That's 48 cookies for Monday plus 24 cookies for Tuesday, yielding a total of 48 + 24 = 72 cookies.", "Hence, on Monday and Tuesday, Marco baked a combined total of 72 cookies."]
        }}

        {{
        "original_question": {question},
        "original_steps": {steps}
        }}
        '''
    
    reworded_json = gpt_query_json(reword_prompt)
    reworded_dict = json.loads(reworded_json)
    print("Original Question: ", question)
    print("Reworded Question: ", reworded_dict["reworded_question"])

    assert len(steps) == len(reworded_dict['reworded_steps']), "Reworded steps not same length!"

    for i, step in enumerate(steps):
        print("Original Step: ", step)
        print("Reworded Step: ", reworded_dict["reworded_steps"][i])

    print()

Original Question:  James creates a media empire.  He creates a movie for $2000.  Each DVD cost $6 to make.  He sells it for 2.5 times that much.  He sells 500 movies a day for 5 days a week.  How much profit does he make in 20 weeks?
Reworded Question:  Lily starts a gardening business. She plants a garden for $2000. Each plant pot costs $6 to prepare. She sells them for 2.5 times that amount. She sells 500 plant pots a day for 5 days a week. What is Lily's profit after 20 weeks?
Original Step:  First, let's calculate the cost of creating the movie. James creates a movie for $2000.
Reworded Step:  First, we calculate the initial investment for planting the garden. Lily invests $2000 to start her gardening business.
Original Step:  Next, let's calculate the cost of making each DVD. Each DVD costs $6 to make.
Reworded Step:  Next, we determine the cost to prepare each plant pot. It costs Lily $6 to prepare each pot.
Original Step:  Now, let's calculate the selling price of each DVD. Jam

In [35]:
import ast
df = pd.read_csv('../embeddings/data/113_gsm8k_gpt35_cot_responses.csv')
df['CoT Steps'] = df['CoT Steps'].apply(ast.literal_eval)
df['Paraphrased CoT Steps'] = df['Paraphrased CoT Steps'].apply(ast.literal_eval)

In [11]:
df.split.value_counts()

split
train    90
val      23
Name: count, dtype: int64

In [29]:
old_df = pd.read_csv('data/gsm8k_cl_trans_para0_47_autosplit.csv')
old_df.head()

,Problem Number,Prompt,Original CoT,Changed Solution,Reasoning Trace,Positive,Negative,split
0,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",To find out how many clips Mohammad sold in Ma...,"[""To ascertain the quantity of clips Mohammad ...",['Step 1: Calculate the number of tennis balls...,train
1,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",To find out how many clips he sold altogether ...,['To ascertain the total number of clips sold ...,['Step 3: Calculate the number of files Mohamm...,train
2,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...","Therefore, Mohammad sold a total of 72 clips i...","['Consequently, the combined number of clips M...","[""Finally, let's calculate how much more money...",train
3,2,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","[""To ascertain Mohammad's income, we must figu...",['Step 2: Convert liters to milliliters. 1 lit...,train
4,2,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","Now, let's multiply his earnings per minute by...","[""Let's proceed to calculate his income by mul...",['Step 3: Calculate the number of files Mohamm...,train


In [36]:
rows = []
for _, row in df.iterrows():
    for step, paraphrase in zip(row['CoT Steps'], row['Paraphrased CoT Steps']):
        new_row = row.to_dict()
        new_row['Reasoning Trace'] = step
        new_row['Positive'] = [paraphrase]
        rows.append(new_row)

df_exploded = pd.DataFrame(rows)

In [42]:
df_exploded.head()

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,Reasoning Trace,Positive
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...","First, let's find out how many clips Natalia s...","[First, we need to figure out the number of se..."
1,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...",To find out how many clips Natalia sold altoge...,[To determine the total number of seashells De...
2,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...","Therefore, Natalia sold a total of 72 clips in...","[In conclusion, Derek collected a combined sum..."
3,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","[To find out how much Weng earned, we need to ...",train,Liam makes $12 per hour tutoring. He tutored f...,"[First, we need to figure out Liam's earnings ...","To find out how much Weng earned, we need to c...","[First, we need to figure out Liam's earnings ..."
4,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","[To find out how much Weng earned, we need to ...",train,Liam makes $12 per hour tutoring. He tutored f...,"[First, we need to figure out Liam's earnings ...","Now, we can multiply the minute rate by the nu...","[Next, we calculate Liam's total earnings by m..."


In [44]:
df_exploded = df_exploded.drop(['CoT Response', 'CoT Sentences'], axis=1)

In [54]:
for idx, row in df_exploded.sample(1).iterrows():
    print('Original: ')
    print(row['Reasoning Trace'])
    print('Paraphrase: ')
    print(row['Positive'])
    # print('Negative Examples:')
    # print(row['Negative'])
    print()

Original: 
First, let's calculate the number of points James scores per game. He scores 4 touchdowns per game, and each touchdown is worth 6 points, so he scores 4 * 6 = <<4*6=24>>24 points per game.
Paraphrase: 
['First, we need to calculate the points Ava scores in each game. She makes 4 baskets per game, with each basket being 6 points, so her points per game are 4 * 6 = 24 points.']



In [55]:
train_df = df_exploded.loc[df_exploded.split == 'train']
val_df = df_exploded.loc[df_exploded.split == 'val']

In [56]:
train_df

,Problem Number,Question,Answer,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,Reasoning Trace,Positive
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...","First, let's find out how many clips Natalia s...","[First, we need to figure out the number of se..."
1,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...",To find out how many clips Natalia sold altoge...,[To determine the total number of seashells De...
2,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...","Therefore, Natalia sold a total of 72 clips in...","[In conclusion, Derek collected a combined sum..."
3,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"[To find out how much Weng earned, we need to ...",train,Liam makes $12 per hour tutoring. He tutored f...,"[First, we need to figure out Liam's earnings ...","To find out how much Weng earned, we need to c...","[First, we need to figure out Liam's earnings ..."
4,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"[To find out how much Weng earned, we need to ...",train,Liam makes $12 per hour tutoring. He tutored f...,"[First, we need to figure out Liam's earnings ...","Now, we can multiply the minute rate by the nu...","[Next, we calculate Liam's total earnings by m..."
...,...,...,...,...,...,...,...,...,...
384,88,Angela is a bike messenger in New York. She ne...,Let p be the number of packages Angela deliver...,"[First, let's assume that Angela delivers x me...",train,Oliver is a librarian in a small town. He need...,"[First, we assume Oliver shelves x magazines. ...","Dividing both sides by 9, we find: x = 3",[Dividing both sides of the equation by 9 give...
385,88,Angela is a bike messenger in New York. She ne...,Let p be the number of packages Angela deliver...,"[First, let's assume that Angela delivers x me...",train,Oliver is a librarian in a small town. He need...,"[First, we assume Oliver shelves x magazines. ...","Therefore, Angela delivers 3 meals.","[Hence, Oliver shelves 3 magazines.]"
386,89,Keiko sent 111 text messages last week. This w...,Last week = 111 texts\nThis week = (2 * 111) -...,"[First, let's find out how many text messages ...",train,Carlos planted 111 trees in his orchard last s...,"[First, we need to figure out the number of tr...","First, let's find out how many text messages K...","[First, we need to figure out the number of tr..."
387,89,Keiko sent 111 text messages last week. This w...,Last week = 111 texts\nThis week = (2 * 111) -...,"[First, let's find out how many text messages ...",train,Carlos planted 111 trees in his orchard last s...,"[First, we need to figure out the number of tr...",To find the total number of text messages Keik...,[To calculate the combined total of trees plan...


In [63]:
val_df.index[-1]

497

In [67]:
train_df['Negative'] = None

In [68]:
import random
from tqdm import tqdm
cur_df = train_df

def sampleRandom(exclude, min_val=0, max_val=125, num_sample=10):
    indices = []
    while len(indices) < num_sample:
        num = random.randint(min_val, max_val)
        if num != exclude and num not in indices:
            indices.append(num)
    assert len(indices) == 10, "wrong number sampled"
    return indices

Processing Rows: 100%|██████████| 389/389 [00:00<00:00, 12845.91it/s]


In [71]:
random.seed(42)

cur_df = train_df
cur_df['Negative'] = None

for i, row in tqdm(cur_df.iterrows(), total = len(cur_df), desc='Processing Rows'):
    step = row['Reasoning Trace']
    negativeIndices = sampleRandom(i, cur_df.index[0], cur_df.index[-1], 10)
    negativeExamples = []
    for j in negativeIndices:
        negativeExamples.append(cur_df['Reasoning Trace'][j])        
    cur_df.at[i, 'Negative'] = negativeExamples

cur_df = val_df
cur_df['Negative'] = None

for i, row in tqdm(cur_df.iterrows(), total = len(cur_df), desc='Processing Rows'):
    step = row['Reasoning Trace']
    negativeIndices = sampleRandom(i, cur_df.index[0], cur_df.index[-1], 10)
    negativeExamples = []
    for j in negativeIndices:
        negativeExamples.append(cur_df['Reasoning Trace'][j])        
    cur_df.at[i, 'Negative'] = negativeExamples

Processing Rows: 100%|██████████| 109/109 [00:00<00:00, 16346.51it/s]


In [75]:
for idx, row in train_df.sample(1).iterrows():
    print('Original: ', idx)
    print(row['Reasoning Trace'])
    print('Paraphrase: ')
    print(row['Positive'])
    for neg_ex in row['Negative']:
        print('Negative Example:', train_df[train_df['Reasoning Trace'] == neg_ex].index[0])
        print(neg_ex)
    print()

Original:  272
Step 6: Calculate the number of sentences Janice started with today. The paper had 536 sentences by the end of today, so Janice started with 536 - 278 = 258 sentences today.
Paraphrase: 
['Step 6: Calculate the number of boxes Carlos had already packed at the start of his shift. With 536 boxes ready by the end of the shift, Carlos must have started with 536 - 278 = 258 boxes at the beginning of his shift today.']
Negative Example: 146
Step 6: Her dog shredded all the money in her wallet except for $1. Therefore, Leah lost $12 - $1 = $11.
Negative Example: 330
Step 3: He also gave away 30 seashells to his brothers, so he had 140 - 30 = 110 seashells left.
Negative Example: 30
First, let's calculate the total amount Alexis spent on the clothes she bought. She spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. So, the total amount she spent on clothes is $30 + $46 + $38 + $11 + $18 = $143.
Negative Example: 45
Step 3: Add

In [76]:
old_df

,Problem Number,Prompt,Original CoT,Changed Solution,Reasoning Trace,Positive,Negative,split
0,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",To find out how many clips Mohammad sold in Ma...,"[""To ascertain the quantity of clips Mohammad ...",['Step 1: Calculate the number of tennis balls...,train
1,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",To find out how many clips he sold altogether ...,['To ascertain the total number of clips sold ...,['Step 3: Calculate the number of files Mohamm...,train
2,1,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...","Therefore, Mohammad sold a total of 72 clips i...","['Consequently, the combined number of clips M...","[""Finally, let's calculate how much more money...",train
3,2,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","[""To ascertain Mohammad's income, we must figu...",['Step 2: Convert liters to milliliters. 1 lit...,train
4,2,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...","Now, let's multiply his earnings per minute by...","[""Let's proceed to calculate his income by mul...",['Step 3: Calculate the number of files Mohamm...,train
...,...,...,...,...,...,...,...,...
120,46,It's Mohammad's birthday party. His parents bo...,Sure! Let's break it down step by step.\n\nFir...,Sure! Let's calculate the cost of the treats f...,"Therefore, the unicorn piñata and the treats c...","['Consequently, the combined price of the unic...","[""Next, let's calculate how much money Mohamma...",val
121,47,Mohammad practices the piano for 20 minutes a ...,"First, let's calculate how long Mohammad pract...",Step 1: Find out how long Mohammad practices t...,"First, let's calculate how long Mohammad pract...","[""Let's figure out the duration of Mohammad's ...","[""Finally, let's calculate the profit made in ...",val
122,47,Mohammad practices the piano for 20 minutes a ...,"First, let's calculate how long Mohammad pract...",Step 1: Find out how long Mohammad practices t...,"Next, let's calculate how long Mohammad practi...","[""Let's proceed to determine the duration of M...","[""Finally, let's calculate how much more money...",val
123,47,Mohammad practices the piano for 20 minutes a ...,"First, let's calculate how long Mohammad pract...",Step 1: Find out how long Mohammad practices t...,"Finally, let's calculate how long Mohammad pra...","[""To conclude, we should work out the duration...","[""Next, let's calculate how long Mohammad prac...",val


In [78]:
full_dataset = pd.concat([train_df, val_df], axis=0)

In [81]:
full_dataset = full_dataset.drop(['Paraphrased CoT Steps', 'CoT Steps'], axis=1)

In [87]:
full_dataset['Positive'] = full_dataset['Positive'].apply(lambda x: [x])

In [89]:
full_dataset['Positive']

0      [First, we need to figure out the number of se...
1      [To determine the total number of seashells De...
2      [In conclusion, Derek collected a combined sum...
3      [First, we need to figure out Liam's earnings ...
4      [Next, we calculate Liam's total earnings by m...
                             ...                        
493    [Thus, the combined length of all Oliver's rop...
494    [Step 1: Determine the total number of fresh a...
495    [Step 2: Figure out the total number of fresh ...
496    [Step 3: Assess the number of fresh apples in ...
497    [Step 4: Calculate the total number of fresh a...
Name: Positive, Length: 498, dtype: object

In [88]:
full_dataset.to_csv('data/gsm8k_cl_trans_para1_112_autosplit.csv')

New Dataset

In [1]:
import pandas as pd
import ast
df = pd.read_csv('data/gsm8k_cl_trans_para1_112_autosplit.csv')
df['']
df

,Unnamed: 0,Problem Number,Question,Answer,split,Paraphrased Question,Reasoning Trace,Positive,Negative
0,0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,train,Derek collected seashells from 48 of his frien...,"First, let's find out how many clips Natalia s...","['First, we need to figure out the number of s...","[""Since we can't have a fraction of a customer..."
1,1,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,train,Derek collected seashells from 48 of his frien...,To find out how many clips Natalia sold altoge...,['To determine the total number of seashells D...,"[""Therefore, Benedict's house is 2350 square f..."
2,2,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,train,Derek collected seashells from 48 of his frien...,"Therefore, Natalia sold a total of 72 clips in...","['In conclusion, Derek collected a combined su...","['Therefore, the unicorn piñata and the treats..."
3,3,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,train,Liam makes $12 per hour tutoring. He tutored f...,"To find out how much Weng earned, we need to c...","[""First, we need to figure out Liam's earnings...",['To find out how much more the cow is worth a...
4,4,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,train,Liam makes $12 per hour tutoring. He tutored f...,"Now, we can multiply the minute rate by the nu...","[""Next, we calculate Liam's total earnings by ...",['Step 2: She spent a seventh of it on a milks...
...,...,...,...,...,...,...,...,...,...
493,493,111,Christina has 3 snakes. 1 snake is 2 feet long...,The first snake is 24 inches because there are...,val,Oliver has 3 ropes. 1 rope is 2 feet long. Ano...,"Therefore, all of Christina's snakes combined ...","[""Thus, the combined length of all Oliver's ro...","[""First, let's calculate the total number of g..."
494,494,112,Madeline has 5 boxes with 24 crayons in each. ...,Two boxes of crayons have 24 x 5/8 = <<24*5/8=...,val,Oliver has 5 baskets with 24 apples in each. H...,Step 1: Calculate the total number of crayons ...,['Step 1: Determine the total number of fresh ...,"['Step 3: On Wednesday, Buddy buys 12 baseball..."
495,495,112,Madeline has 5 boxes with 24 crayons in each. ...,Two boxes of crayons have 24 x 5/8 = <<24*5/8=...,val,Oliver has 5 baskets with 24 apples in each. H...,Step 2: Calculate the total number of crayons ...,['Step 2: Figure out the total number of fresh...,"[""Finally, let's calculate the total amount of..."
496,496,112,Madeline has 5 boxes with 24 crayons in each. ...,Two boxes of crayons have 24 x 5/8 = <<24*5/8=...,val,Oliver has 5 baskets with 24 apples in each. H...,Step 3: Calculate the number of unused crayons...,"[""Step 3: Assess the number of fresh apples in...","[""First, let's calculate the total cost of the..."
